In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.0-preview2/spark-3.0.0-preview2-bin-hadoop3.2.tgz
!tar -xvf spark-3.0.0-preview2-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark

spark-3.0.0-preview2-bin-hadoop3.2/
spark-3.0.0-preview2-bin-hadoop3.2/data/
spark-3.0.0-preview2-bin-hadoop3.2/data/streaming/
spark-3.0.0-preview2-bin-hadoop3.2/data/streaming/AFINN-111.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/sample_binary_classification_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/sample_kmeans_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/sample_multiclass_classification_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/sample_lda_libsvm_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/iris_libsvm.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/pagerank_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/sample_linear_regression_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/pic_data.txt
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/als/
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/als/test.data
spark-3.0.0-preview2-bin-hadoop3.2/data/mllib/als/sample_movielens_rati

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

import collections

In [4]:
from google.colab import files
files.upload()

Saving fakefriends.csv to fakefriends.csv


{'fakefriends.csv': b'0,Will,33,385\r\n1,Jean-Luc,26,2\r\n2,Hugh,55,221\r\n3,Deanna,40,465\r\n4,Quark,68,21\r\n5,Weyoun,59,318\r\n6,Gowron,37,220\r\n7,Will,54,307\r\n8,Jadzia,38,380\r\n9,Hugh,27,181\r\n10,Odo,53,191\r\n11,Ben,57,372\r\n12,Keiko,54,253\r\n13,Jean-Luc,56,444\r\n14,Hugh,43,49\r\n15,Rom,36,49\r\n16,Weyoun,22,323\r\n17,Odo,35,13\r\n18,Jean-Luc,45,455\r\n19,Geordi,60,246\r\n20,Odo,67,220\r\n21,Miles,19,268\r\n22,Quark,30,72\r\n23,Keiko,51,271\r\n24,Julian,25,1\r\n25,Ben,21,445\r\n26,Julian,22,100\r\n27,Leeta,42,363\r\n28,Martok,49,476\r\n29,Nog,48,364\r\n30,Keiko,50,175\r\n31,Miles,39,161\r\n32,Nog,26,281\r\n33,Dukat,53,197\r\n34,Jean-Luc,43,249\r\n35,Beverly,27,305\r\n36,Kasidy,32,81\r\n37,Geordi,58,21\r\n38,Deanna,64,65\r\n39,Morn,31,192\r\n40,Odo,52,413\r\n41,Hugh,67,167\r\n42,Brunt,54,75\r\n43,Guinan,58,345\r\n44,Nerys,35,244\r\n45,Dukat,52,77\r\n46,Morn,25,96\r\n47,Brunt,24,49\r\n48,Nog,20,1\r\n49,Ezri,40,254\r\n50,Quark,51,283\r\n51,Lwaxana,36,212\r\n52,Beverly,19,269\

In [5]:
# Create a SparkSession (Note, the config section is only for Windows!)
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [6]:
def mapper(line):
    fields = line.split(',')
    return Row(ID=int(fields[0]), name=str(fields[1].encode("utf-8")), age=int(fields[2]), numFriends=int(fields[3]))

In [7]:
lines = spark.sparkContext.textFile("fakefriends.csv")
people = lines.map(mapper)


In [8]:
# Infer the schema, and register the DataFrame as a table.
schemaPeople = spark.createDataFrame(people).cache()
schemaPeople.createOrReplaceTempView("people")

In [9]:
# SQL can be run over DataFrames that have been registered as a table.
teenagers = spark.sql("SELECT * FROM people WHERE age >= 13 AND age <= 19")

In [10]:
for teen in teenagers.collect():
  print(teen)

Row(ID=21, name="b'Miles'", age=19, numFriends=268)
Row(ID=52, name="b'Beverly'", age=19, numFriends=269)
Row(ID=54, name="b'Brunt'", age=19, numFriends=5)
Row(ID=106, name="b'Beverly'", age=18, numFriends=499)
Row(ID=115, name="b'Dukat'", age=18, numFriends=397)
Row(ID=133, name="b'Quark'", age=19, numFriends=265)
Row(ID=136, name="b'Will'", age=19, numFriends=335)
Row(ID=225, name="b'Elim'", age=19, numFriends=106)
Row(ID=304, name="b'Will'", age=19, numFriends=404)
Row(ID=341, name="b'Data'", age=18, numFriends=326)
Row(ID=366, name="b'Keiko'", age=19, numFriends=119)
Row(ID=373, name="b'Quark'", age=19, numFriends=272)
Row(ID=377, name="b'Beverly'", age=18, numFriends=418)
Row(ID=404, name="b'Kasidy'", age=18, numFriends=24)
Row(ID=409, name="b'Nog'", age=19, numFriends=267)
Row(ID=439, name="b'Data'", age=18, numFriends=417)
Row(ID=444, name="b'Keiko'", age=18, numFriends=472)
Row(ID=492, name="b'Dukat'", age=19, numFriends=36)
Row(ID=494, name="b'Kasidy'", age=18, numFriends=194)

In [11]:
spark.stop()